In [2]:
import pickle
import argparse
import xgboost as xgb
import multiprocessing
import pandas as pd
from utils import *

In [3]:
import matplotlib as mpl
import pylab as plt
%matplotlib inline 

mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

In [39]:

wiki_id = "simplewiki"
data = "../../data/{0}/testing/{0}.backtest.eval.csv".format(wiki_id)
df=pd.read_csv(data,index_col=0).round(3)
print(df.T)

                         0          1          2          3          4  \
threshold            0.000      0.100      0.200      0.300      0.400   
N                10000.000  10000.000  10000.000  10000.000  10000.000   
micro_precision      0.547      0.602      0.655      0.707      0.753   
micro_recall         0.643      0.639      0.622      0.584      0.536   

                         5          6          7          8          9  
threshold            0.500      0.600      0.700      0.800      0.900  
N                10000.000  10000.000  10000.000  10000.000  10000.000  
micro_precision      0.795      0.840      0.877      0.910      0.961  
micro_recall         0.471      0.387      0.296      0.191      0.074  


In [40]:
df.T

,0,1,2,3,4,5,6,7,8,9
threshold,0.000,0.100,0.200,0.300,0.400,0.500,0.600,0.700,0.800,0.900
N,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000
micro_precision,0.547,0.602,0.655,0.707,0.753,0.795,0.840,0.877,0.910,0.961
micro_recall,0.643,0.639,0.622,0.584,0.536,0.471,0.387,0.296,0.191,0.074


In [41]:
df_all = pd.DataFrame()
list_wiki_id = ["arwiki","bnwiki","cswiki","viwiki","dewiki","ptwiki","simplewiki"]
for wiki_id in list_wiki_id:
    data = "../../data/{0}/testing/{0}.backtest.eval.csv".format(wiki_id)
    df=pd.read_csv(data,index_col=0)
    df['F1'] = 2*(df['micro_precision']*df['micro_recall'])/(df['micro_precision']+df['micro_recall'])
    df = df.round(3)
    df=df.drop(columns=['N','threshold'])
    df = df.rename(columns = {
        "micro_precision": "Pre.",
        "micro_recall": "Rec.",
    })
    
    print(wiki_id)
    df_all = df_all.append(df.T)
    print(df.T)

arwiki
          0      1      2      3      4      5      6      7      8      9
Pre.  0.515  0.574  0.623  0.671  0.715  0.754  0.797  0.845  0.907  0.955
Rec.  0.449  0.451  0.441  0.422  0.391  0.349  0.299  0.228  0.145  0.076
F1    0.480  0.505  0.516  0.518  0.506  0.477  0.435  0.359  0.250  0.141
bnwiki
          0      1      2      3      4      5      6      7      8      9
Pre.  0.563  0.588  0.623  0.657  0.696  0.743  0.801  0.856  0.903  0.956
Rec.  0.436  0.431  0.421  0.397  0.361  0.297  0.221  0.147  0.090  0.033
F1    0.491  0.497  0.503  0.495  0.476  0.425  0.346  0.250  0.163  0.064
cswiki
          0      1      2      3      4      5      6      7      8      9
Pre.  0.601  0.645  0.677  0.710  0.740  0.778  0.826  0.866  0.910  0.953
Rec.  0.579  0.571  0.552  0.528  0.487  0.430  0.359  0.269  0.171  0.058
F1    0.590  0.606  0.608  0.605  0.588  0.554  0.500  0.410  0.287  0.110
viwiki
          0      1      2      3      4      5      6      7      8     

In [42]:
df_all

,0,1,2,3,4,5,6,7,8,9
Pre.,0.515,0.574,0.623,0.671,0.715,0.754,0.797,0.845,0.907,0.955
Rec.,0.449,0.451,0.441,0.422,0.391,0.349,0.299,0.228,0.145,0.076
F1,0.480,0.505,0.516,0.518,0.506,0.477,0.435,0.359,0.250,0.141
Pre.,0.563,0.588,0.623,0.657,0.696,0.743,0.801,0.856,0.903,0.956
Rec.,0.436,0.431,0.421,0.397,0.361,0.297,0.221,0.147,0.090,0.033
F1,0.491,0.497,0.503,0.495,0.476,0.425,0.346,0.250,0.163,0.064
Pre.,0.601,0.645,0.677,0.710,0.740,0.778,0.826,0.866,0.910,0.953
Rec.,0.579,0.571,0.552,0.528,0.487,0.430,0.359,0.269,0.171,0.058
F1,0.590,0.606,0.608,0.605,0.588,0.554,0.500,0.410,0.287,0.110
Pre.,0.661,0.756,0.803,0.837,0.870,0.903,0.932,0.958,0.982,0.995
